<a href="https://colab.research.google.com/github/saravananpsg/nlp/blob/master/ray_distributed_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 33.9 MB/s eta 0:00:00


In [9]:
import wandb
wandb.Api()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sarapsg to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [28]:
import torch
import subprocess
import socket
import time
import ray
import wandb
from datasets import load_dataset, DatasetDict
from transformers import (AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments,
                          DataCollatorForLanguageModeling)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

# Model and Paths
MODEL_NAME = "meta-llama/Llama-2-7b-hf"  # Replace with your model
LORA_SAVE_PATH = "./lora_adapter"
MERGED_MODEL_PATH = "./merged_llm"

# ==============================
# 1️⃣ Start Ray Cluster
# ==============================

def get_local_ip():
    """Get the local IP address of the machine."""
    return socket.gethostbyname(socket.gethostname())

def start_ray_cluster():
    """Start Ray head node or connect worker nodes dynamically."""
    local_ip = get_local_ip()
    try:
        # Try starting Ray as head node
        print("Starting Ray Head Node...")
        subprocess.run(["ray", "start", "--head"], check=True)
        print("Ray Head Node started successfully!")
    except subprocess.CalledProcessError:
        print("Ray Head Node is already running, assuming this is a worker node.")

        # Connect worker node to head node
        head_node_ip = local_ip  # You can pass this as an environment variable
        print(f"Connecting to Ray Head Node at {head_node_ip}...")
        subprocess.run(["ray", "start", f"--address={head_node_ip}:6379"], check=True)

# Start Ray Cluster
start_ray_cluster()

# Initialize Ray
ray.init(ignore_reinit_error=True)

# ==============================
# 2️⃣ Load IMDb Dataset
# ==============================

def load_and_preprocess_data():
    """Load and preprocess IMDb dataset for causal LM training."""
    dataset = load_dataset("imdb")

    # Combine text and label into a single format
    def format_text(example):
        return {"text": f"Review: {example['text']}\nSentiment: {'positive' if example['label'] == 1 else 'negative'}"}

    dataset = dataset.map(format_text, batched=False)

    # Train/Test split (keeping only text column)
    dataset = DatasetDict({
        "train": dataset["train"].remove_columns(["label"]),
        "test": dataset["test"].remove_columns(["label"]),
    })

    return dataset

# ==============================
# 3️⃣ LoRA Training Function
# ==============================

@ray.remote(num_gpus=1)
def train_lora():
    wandb.init(project="IMDb_LoRA", entity="sarapsg")  # Replace with your WandB details

    # Load dataset
    dataset = load_and_preprocess_data()

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, load_in_8bit=True, device_map="auto")

    # LoRA Configuration
    lora_config = LoraConfig(
        r=16, lora_alpha=32,
        lora_dropout=0.1, bias="none"
    )
    model = get_peft_model(model, lora_config)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=True,
        logging_dir="./logs",
        remove_unused_columns=False,
    )

    # Tokenize the dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Data collator
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    # Train the model
    trainer.train()

    # Save LoRA adapters
    model.save_pretrained(LORA_SAVE_PATH)
    tokenizer.save_pretrained(LORA_SAVE_PATH)

    # Extract final loss
    final_loss = trainer.state.log_history[-1]['loss'] if trainer.state.log_history else None

    return final_loss

# Run LoRA Training on Multiple GPUs
losses = ray.get([train_lora.remote() for _ in range(2)])  # Adjust workers as needed
print("Final Training Losses from Parallel Runs:", losses)

# ==============================
# 4️⃣ Merge LoRA Weights
# ==============================

@ray.remote(num_gpus=1)
def merge_lora():
    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

    # Load trained LoRA adapter
    lora_model = PeftModel.from_pretrained(base_model, LORA_SAVE_PATH)

    # Merge LoRA weights into base model
    merged_model = lora_model.merge_and_unload()

    # Save merged model
    merged_model.save_pretrained(MERGED_MODEL_PATH)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.save_pretrained(MERGED_MODEL_PATH)

    print(f"Merged model saved at {MERGED_MODEL_PATH}")

# Run Merging on Distributed GPUs
ray.get(merge_lora.remote())

# ==============================
# 5️⃣ Inference with Merged Model
# ==============================

def run_inference():
    print("Loading merged model for inference...")
    merged_pipeline = pipeline("text-generation", model=MERGED_MODEL_PATH)

    # Generate text
    output = merged_pipeline("The movie was", max_length=50)
    print("Generated Text:", output)

run_inference()


Starting Ray Head Node...


 12%|█▏        | 390/3125 [04:45<33:18,  1.37it/s]


Ray Head Node is already running, assuming this is a worker node.
Connecting to Ray Head Node at 172.28.0.12...


 13%|█▎        | 392/3125 [04:47<33:20,  1.37it/s]
2025-03-20 04:33:01,820	INFO worker.py:1654 -- Connecting to existing Ray cluster at address: 172.28.0.12:6379...
2025-03-20 04:33:01,821	INFO worker.py:1672 -- Calling ray.init() again after it has already been called.
(pid=4194) 2025-03-20 03:50:15.849869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=4194) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=4194) E0000 00:00:1742442615.872188    4194 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=4194) E0000 00:00:1742442615.878946    4194 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(train_lora pid=4194) The `load_in_

(train_lora pid=17751) {'train_runtime': 2.6691, 'train_samples_per_second': 1.124, 'train_steps_per_second': 1.124, 'train_loss': 4.042674700419108, 'mean_token_accuracy': 0.2777777810891469, 'epoch': 1.0}
(merge_lora pid=17750) Merged model saved at ./merged_llm


 13%|█▎        | 399/3125 [04:53<36:11,  1.26it/s]
(pid=4193) 2025-03-20 04:01:13.529453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered [repeated 22x across cluster]
(pid=4193) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 22x across cluster]
(pid=4193) E0000 00:00:1742443273.552946    4193 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered [repeated 22x across cluster]
(pid=4193) E0000 00:00:1742443273.559851    4193 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered [repeated 22x across cluster]
(train_lora pid=4193) The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please

RayTaskError(ValueError): [36mray::train_lora()[39m (pid=22919, ip=172.28.0.12)
  File "<ipython-input-28-8fa0fd3224c3>", line 84, in train_lora
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py", line 4188, in from_pretrained
    hf_quantizer.validate_environment(device_map=device_map)
  File "/usr/local/lib/python3.11/dist-packages/transformers/quantizers/quantizer_bnb_8bit.py", line 101, in validate_environment
    raise ValueError(
ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`

Map: 100%|█████████▉| 49849/50000 [00:03<00:00, 16471.84 examples/s]